In the name of God  

# RWP

In [1]:
#Import library 
from pymobility.models.mobility import random_waypoint

In [60]:
rw = random_waypoint(12, dimensions=(500, 500), velocity=(5.0, 5.0), wt_max=1000.0)

In [ ]:
positions = next(rw)
positions

In [ ]:
for positions in rw:
    print(positions)

In [62]:
import numpy as np
from math import log as ln
from math import sin,cos,pi,sqrt,atan,log2,exp,pow
class LiFiAccessPoint:
    def __init__(self,x,y):
        self.x      = x                 # horizontal position of LiFi AP
        self.y      = y                 # vertical position of LiFi AP
        self.N_LiFi = pow(10,(-21))     # LiFi noise power spectral density 
        self.B_LiFi = 40*10e6           # LiFi bandwidth (Hz)
        self.P_opt = 3                  # Optical power (watt)
        self.R     = 0.53               # Responsivity (A/W)
        self.scale = 100
    def calculate_distance(self,user_position):
        return sqrt( pow((self.x-user_position[0]),2) + pow((self.y-user_position[1]),2) ) / self.scale


    def phi(self,user_position): # Angle between user-to-lifi access point with room vertical orthogonal line
        h = 2                                       # Room height (m)
        d = self.calculate_distance(user_position)
        return atan( d / h)
    
    def g_c(self,psi): # Gain of optical concentrator
        self.PSI_MAX    = pi/4  # Semi-angle of field of view (radian)
        n               = 1.5   # Refractive index
        if 0 <= psi and psi <= self.PSI_MAX:
            return pow(n,2)/pow(sin(psi),2)
        else:
            return 0

    def H_LOS(self,user_position): # Line of sight channel coefficient
        led_half_pi= pi/3                                   # LED field of view (radian)
        m          = -ln(2)/ln(cos(led_half_pi))            # lambertian order
        A_PD       = 1                                      # Area of photodetector (cm^2)
        g_f        = 1                                      # Optical filter gain
        phi        = self.phi(user_position=user_position)
        g_c        = self.g_c
        d          = self.calculate_distance(user_position)
        return ( (m+1)*A_PD*cos(phi)*g_f*g_c(phi)*cos(phi) ) / (2*pi*pow(d,2))

    def H_LiFi(self,user):
        return self.H_LOS(user.position) 

    def SNR(self,user): # Signal to noise ratio
        return ( pow((self.H_LiFi(user)*self.P_opt*self.R),2) ) / (self.N_LiFi*self.B_LiFi)

    def SINR(self,user,other_LiFi_AP): # Signal to interference noise ratio 
        interference = 0 
        for i in other_LiFi_AP: 
            interference += i.H_LiFi(user)*i.P_opt*i.R
        return ( pow((self.H_LiFi(user)*self.P_opt*self.R),2) ) / (self.N_LiFi*self.B_LiFi + interference)


    def r(self,user,other_LiFi_AP): # Achievable rate
        return (self.B_LiFi/2)*log2(1+(6/(pi*exp(1)))*self.SINR(user,other_LiFi_AP)) 

class WiFiAccessPoint:
    def __init__(self,x,y):
        self.x = x
        self.y = y


class User:
    def __init__(self,number,position):
        self.position = position
        self.number = number

In [63]:
from tkinter import *
from time import time, sleep
from PIL import Image,ImageTk
import math
import functools
canvas_width            = 500
canvas_height           = 500
actual_room_width       = 5 #(m)
actual_room_height      = 5 #(m)
main_window_resolution  = "550x550"
scale = 500 / 500

# dimension of users
rectangle_width = 10
rectangle_height = 10


N_USER          = 12                                             # Number of users
N_LiFi_AP       = 4                                             # Number of LiFi access point
N_WiFi_AP       = 1                                             # Number of WiFi access point


LiFi_AP_coordinate = [[125,125],[125,375],[375,125],[375,375]] 
# def paint(event):
#     python_green = "#476042"
#     x1, y1 = (event.x - 1), (event.y - 1)
#     x2, y2 = (event.x + 1), (event.y + 1)
#     w.create_oval(x1, y1, x2, y2, fill=python_green)

def showMovement():
        global position

        for i in range(N_USER):
                x1 = position[i][0]
                y1 = position[i][1]
                position=next(rw)
                x2 = position[i][0]
                y2 = position[i][1]
                dx = x2 - x1
                dy = y2 - y1
                w.move(users[i],dx*scale,dy*scale)
                w.move(user_labels[i],dx*scale,dy*scale)
                user_objects[i].position = w.coords(users[i])
        calculate_snr(LiFi_APs,[],user_objects)
        
        w.after(50,showMovement)


def calculate_snr(LiFi_APs,WiFi_APs,users):
        global SNRs

        SNRs = np.zeros((N_USER,N_LiFi_AP+N_WiFi_AP))
        
        for i in range(N_USER):
                for j in range(N_LiFi_AP):
                        other_LiFi_AP = [value["LiFi_AP_obj"] for index, value in enumerate(LiFi_APs) if index != j]
                        SNRs[i][j] = LiFi_APs[j]["LiFi_AP_obj"].SINR(user_objects[i],other_LiFi_AP)
                w.itemconfig(user_labels[i], text=np.argmax(SNRs[i]), font=('Helvetica 8 bold'),fill="white")
                
       

master = Tk()
# master = Toplevel()
master.geometry(main_window_resolution)
master.title("Room")
master.configure(bg="black")

w = Canvas(master,
           width=canvas_width,
           height=canvas_height,background="black")
w.pack(expand=YES, fill=BOTH)


# w.pack()

position = next(rw)
users=[]
user_objects=[]
user_labels=[]
LiFi_APs=[]
# for n in range(Nu):
#     users.append(w.create_rectangle(position[n][0]*scale,position[n][1]*scale,
#                     position[n][0]*scale+rectangle_width,
#                     position[n][1]*scale+rectangle_height,fill='blue'))

colors=['blue','green','red','yellow','orange','white','blue','green','red','yellow','orange','white']

for n in range(N_USER):
        users.append(w.create_oval(position[n][0]*scale,position[n][1]*scale,
                    position[n][0]*scale+rectangle_width,
                    position[n][1]*scale+rectangle_height,fill=colors[n])
                    )
        user_objects.append(User(n,w.coords(users[n])))
        if(position[n][0]-rectangle_width >= 0 and position[n][1]-rectangle_height >= 0):
                user_labels.append(w.create_text(position[n][0]-rectangle_width, position[n][1]-rectangle_height, text="HELLO WORLD", fill="black", font=('Helvetica 15 bold')))
        else:
                user_labels.append(w.create_text(0, 0, text="", fill="black", font=('Helvetica 5 bold')))

                
LiFi_AP_img = Image.open("./images/LiFiAP.png")
LiFi_AP_img = LiFi_AP_img.resize((45,45),Image.ANTIALIAS)
LiFi_AP_img = ImageTk.PhotoImage(LiFi_AP_img)
scale_x = canvas_width/actual_room_width
scale_y = canvas_width/actual_room_height

for i in range(N_LiFi_AP):
        x = LiFi_AP_coordinate[i][0]
        y = LiFi_AP_coordinate[i][1]
        LiFi_APs.append({"LiFi_AP_image": w.create_image(x,y,anchor=NW,image=LiFi_AP_img) , "LiFi_AP_obj" : LiFiAccessPoint(x,y)})


w.after(50,showMovement)


mainloop()    

In [31]:
from math import log as ln
from math import pi

In [32]:
class test:
    def __init__(self):
        self.m = 3

test().m

3

In [10]:
scale_x

100.0

In [50]:
colors.__len__()

12